In [2]:
 # Dependencies
from bs4 import BeautifulSoup
import requests
import os
import pymongo
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd
from zipfile import ZipFile
from IPython.display import display_html

In [3]:
# set up the splinter method
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# define the target URL
url = 'https://mars.nasa.gov/news'
browser.visit(url)

In [5]:
# select the "title" portion
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

titlePortion = soup.find('div', class_='content_title')
titlePortion

<div class="content_title"><a href="/news/8388/what-two-planetary-siblings-can-teach-us-about-life/" target="_self">What Two Planetary Siblings Can Teach Us About Life</a></div>

In [6]:
# select the "paragraph" portion
parPortion = soup.find('div', class_='article_teaser_body')
parPortion

<div class="article_teaser_body">In studying Mars, NASA's InSight will reveal what makes one planet more or less suitable for life than another.</div>

In [7]:
# use get_text() to only get the needed text
strippedTitle = titlePortion.get_text()
strippedTitle

'What Two Planetary Siblings Can Teach Us About Life'

In [8]:
# use get_text() to only get the needed text
strippedPar = parPortion.get_text()
strippedPar

"In studying Mars, NASA's InSight will reveal what makes one planet more or less suitable for life than another."

In [9]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and... 
# assign the url string to a variable called featured_image_url

# define the target URL
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)

# redefine html and soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [10]:
# search code for image tags
image = soup.find('article', class_="carousel_item")
cutImage = image['style']
cutImage

"background-image: url('/spaceimages/images/wallpaper/PIA17661-1920x1200.jpg');"

In [11]:
# I'm having trouble isolating the part I want, need to know the elements
type(cutImage)

str

In [12]:
# try and get only the URL
featured_image_url = cutImage.split("url")[1]
featured_image_url

"('/spaceimages/images/wallpaper/PIA17661-1920x1200.jpg');"

In [13]:
# Visit the Mars Weather twitter account here and scrape the latest 
# Mars weather tweet from the page. Save the tweet text for the weather 
# report as a variable called mars_weather.

# define the target URL
url3 = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url3)

# redefine html and soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [14]:
# scrape the latest tweet
tweet = soup.find('div', class_='js-tweet-text-container')
tweet

<div class="js-tweet-text-container">
<p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">Sol 2233 (2018-11-17), high -4C/24F, low -72C/-97F, pressure at 8.61 hPa, daylight 06:24-18:41</p>
</div>

In [15]:
# report as a variable called mars_weather.
mars_weather = tweet.find('p').text
mars_weather

'Sol 2233 (2018-11-17), high -4C/24F, low -72C/-97F, pressure at 8.61 hPa, daylight 06:24-18:41'

In [16]:
# Visit the Mars Facts webpage here and use Pandas to scrape the table 
# containing facts about the planet including Diameter, Mass, etc.
url4 = 'https://space-facts.com/mars/'
browser.visit(url4)

# redefine html and soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [17]:
table = soup.find('div')

# find the rows within the table
rows = table.find_all('tr')

rows

[<tr class="row-1 odd">
 <td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/>
 </td>
 </tr>, <tr class="row-2 even">
 <td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/>
 </td>
 </tr>, <tr class="row-3 odd">
 <td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.42 x 10^23 kg (10.7% Earth)</td>
 </tr>, <tr class="row-4 even">
 <td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/deimos/">Deimos</a>)</td>
 </tr>, <tr class="row-5 odd">
 <td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km (1.52 AU)</td>
 </tr>, <tr class="row-6 even">
 <td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/>
 </td>
 </tr>, <tr class="row-7 odd">
 <td class="column-1"><strong>Surface Temperature: </strong></

In [18]:
type(rows)

bs4.element.ResultSet

In [19]:
# find the tds elements within the rows and print out the rows and row values
facts = []
values = []

for row in rows:
    tds = row.find_all('td')
    fact = tds[0].get_text()
    facts.append(fact)
    value = tds[1].get_text()
    values.append(value)
    
#     print(fact)
#     print(value)

print(facts)
print(values)

['Equatorial Diameter:', 'Polar Diameter:', 'Mass:', 'Moons:', 'Orbit Distance:', 'Orbit Period:', 'Surface Temperature: ', 'First Record:', 'Recorded By:']
['6,792 km\n', '6,752 km\n', '6.42 x 10^23 kg (10.7% Earth)', '2 (Phobos & Deimos)', '227,943,824 km (1.52 AU)', '687 days (1.9 years)\n', '-153 to 20 °C', '2nd millennium BC', 'Egyptian astronomers']


In [20]:
# I have two lists, put those into a dataframe
marsFactsTable = pd.DataFrame({
    'facts': facts,
    'values': values
})

marsFactsTable

,facts,values
0,Equatorial Diameter:,"6,792 km\n"
1,Polar Diameter:,"6,752 km\n"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)\n
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [21]:
# read the table into an HTML table
marsFactsTable.to_html("marsFacts.html")

In [30]:
# Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url5)

# redefine html and soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [31]:
# I think I'm looking for 'div' class="item"
# targetImages = soup.find_all('div', class_="item")

# #titles = targetImages.href
# # titles

# for targetImage in targetImages:
#     links = targetImage.find('a')
#     link = links.get("href")
#     print(links)
#     print(link)

In [32]:
# Get the list of links 
listOfLinks = []

testLinks = browser.find_by_tag('h3')
for i in range(len(testLinks)):
    hemisphere = {}
    browser.find_by_tag('h3')[i].click()
    sample = browser.find_link_by_text('Sample').first
    hemisphere['image_url'] = sample['href']
    hemisphere['title'] = browser.find_by_css('h2.title').text
    listOfLinks.append(hemisphere)
    
print(listOfLinks)

[{'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}, {'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}, {'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}]
